In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optmin
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from kan import *
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv("../../../transformed_datasets/transformed_NSL_KDD_2.csv")

In [4]:
selected_features = ['dst_host_same_src_port_rate',
 'is_guest_login',
 'num_failed_logins',
 'dst_host_diff_srv_rate',
 'rerror_rate',
 'logged_in',
 'serror_rate',
 'same_srv_rate',
 'protocol_type_0',
'protocol_type_1',
 'dst_host_srv_diff_host_rate']

In [5]:
feature_columns = []
for feature in df.columns:
    if df[feature].dtype == 'int64' or df[feature].dtype == 'float64':
        feature_columns.append(feature)

In [6]:
features_to_be_normalised = []
for feature in df.columns:
    if feature == 'label':
        continue
    min_val = df[feature].min()
    max_val = df[feature].max()
    if not min_val >= 0.0 and max_val <= 1.0:
        features_to_be_normalised.append(feature)
print(features_to_be_normalised)

[]


In [7]:
# scaler = StandardScaler()
# df[features_to_be_normalised] = scaler.fit_transform(df[features_to_be_normalised])
# df[features_to_be_normalised].describe()
df['label'].nunique()


38

In [ ]:
label_encoder = LabelEncoder()
df['label'].nunique()
df['label'] = label_encoder.fit_transform(df['label'])

In [ ]:
def load_dataset():
    features_tensor = torch.tensor(df[selected_features].values, dtype=torch.float32)
    target_tensor = torch.tensor(df['label'].values.reshape(-1, 1), dtype=torch.long)
    features_tensor.to(dtype=torch.float64)
    target_tensor.to(dtype=torch.float64)
    X_Train, X_Test, y_train, y_test = train_test_split(features_tensor, target_tensor, test_size=0.2, random_state=42)
    train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_Train, y_train), batch_size=1, shuffle=True)
    test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_Test, y_test), batch_size=1, shuffle=False)

    train_inputs = torch.empty(0, 11, device=device)
    train_labels = torch.empty(0, 1, device=device)
    test_inputs = torch.empty(0, 11, device=device)
    test_labels = torch.empty(0, 1, device=device)

    # Concatenate all data into a single tensor on the specified device
    for data, labels in train_loader:
        train_inputs = torch.cat((X_Train, data.to(device)), dim=0)
        train_labels = torch.cat((y_train, labels.to(device)), dim=0)

    for data, labels in test_loader:
        test_inputs = torch.cat((X_Test, data.to(device)), dim=0)
        test_labels = torch.cat((y_test, labels.to(device)), dim=0)
    print(train_inputs, train_labels)
    dataset = {}
    dataset['train_input'] = train_inputs
    dataset['test_input'] = test_inputs
    dataset['train_label'] = train_labels
    dataset['test_label'] = test_labels
    dataset['train_input'] = dataset['train_input']
    dataset['train_label'] = dataset['train_label']

    return dataset

In [ ]:
device = None
if torch.cuda.is_available():
    device = torch.device("mpu")
else:
    device = torch.device("cpu")
device

In [ ]:
dataset = load_dataset()

In [ ]:
model = KAN(width=[11, 32, 16, 23], grid=3, k=3, seed=42)

In [ ]:
model.fit(dataset, opt="LBFGS", steps=100, lamb=0.001);